# Code to analyze the locomotion test

In [ ]:
#DEV
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import tables as tb
import h5py
import scipy.stats
import datetime
from scipy.ndimage.filters import gaussian_filter as smooth
from IPython.display import clear_output, display, HTML, Image
import matplotlib.cm as cm
import warnings
import matplotlib.backends.backend_pdf


warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib.pyplot as plt

if OS() == 'Linux':
    root = "/data"
elif OS() == 'Windows':
    root = "C:\\DATA\\"
else:
    root = "/Users/davidrobbe/Documents/Data/"

param={
    "goalTime":7,#needed for pavel data only
    "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
    "maxTrialDuration":15,
    "interTrialDuration":10,#None pavel
    "endTrial_frontPos":30,
    "endTrial_backPos":55, 
    "endTrial_minTimeSec":4,
    "cameraSamplingRate":25, #needed for new setup    

    "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
    "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
    "nbJumpMax":100,#200 pavel
    "binSize":0.25,
    #parameters used to preprocess (will override the default parameters)
}  
ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
CWD=os.getcwd()
os.chdir(CommonNoteBookesPath)
%run Animal_Tags.ipynb
%run UtilityTools.ipynb
%run BatchRatBehavior.ipynb
%run plotRat_documentation_3_KinematicsInvestigation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run loadRat_documentation.ipynb
%run RunBatchRat_3_CompareGroups.ipynb
%run Lesion_Size.ipynb

os.chdir(CWD)

print('Done!')

In [ ]:
LesionAnimalList=['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                  'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']#, 'Rat329', 'Rat333']
NormalAnimalList=['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                  'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

profile={'Type':'Good',
         'rewardType':'Progressive',
#          'initialSpeed':'10',
#          'Speed':['10'],
         'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP','Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                'Control-EXP','ImmobileTreadmill-EXP']
         }

sessionList0=batch_get_session_list(root,NormalAnimalList,profile)['Sessions']
sessionList1=batch_get_session_list(root,LesionAnimalList,profile)['Sessions']

# Forward Speed Detection

In [ ]:
pos=data.position[17]

cs=int(data.cameraSamplingRate)
MinDuration=1   #s
MinDisplacement=30  #cm

def _forwardRunningSpeed_loco(pos, cs: int, MinDur=1, MinDisplacement=30):
    MinDuration=MinDur*cs
    posLen=len(pos)
    posDiff=np.diff(a=pos, prepend=pos[0])
    ForwardDiff=(posDiff<=0).astype(int)
    edges=ForwardDiff - np.concatenate((np.array([False]),ForwardDiff[:-1]))
    try:
        ForwardEpoch=np.where(edges!=0)[0].reshape(-1,2)
    except ValueError:
        ForwardEpoch=np.where(edges!=0)[0][:-1].reshape(-1,2)

    _ForwardSpeed=[]
    for i,j in ForwardEpoch:
        if (j-i < MinDuration                   #short epoch
            or pos[i]-pos[j] < MinDisplacement  #short displacement
        ):
            continue

        _ForwardSpeed.append(((pos[i]-pos[j])/(i-j))*cs)

    return _ForwardSpeed

def loco_forward_speed (data, trdSpeed):
    cs=int(data.cameraSamplingRate)
    speeds=[]
    
    for trial,pos in data.position.items():
        if data.treadmillSpeed[trial] != trdSpeed:
            continue
        speeds.extend(_forwardRunningSpeed_loco(pos=pos, cs=cs))
        
    return np.nanmedian(speeds)

# Plot / Save the trajectories

In [ ]:
def plot_position_of_trials(data,speed,ax=None):
    assert data.dataType == 'ViewPoint'
    if ax is None:
        plt.figure(figsize=(15,5))
        ax=plt.gca()
    
    for trial,pos in data.position.items():
        if data.treadmillSpeed[trial] != speed or trial < 10:
            continue
        
        time=np.linspace(0,data.maxTrialDuration[trial],len(pos))
        
        ax.plot(time,pos,lw=1)
        
    ax.set_ylim([0,90])
    ax.set_xlim([0,30])
    ax.set_title(f'speed={speed}, {data.animal}')

In [ ]:
_SPD_=(0,10,15,20,25,30,35,40)
pdf = matplotlib.backends.backend_pdf.PdfPages("/home/david/Documents/TEMP/Control.pdf")
#Control
for i,session in enumerate(sessionList0):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    fig=plt.figure(figsize=(15,5*len(_SPD_)))
    for i,speed in enumerate(_SPD_):
        ax=fig.add_subplot(len(_SPD_),1,i+1)
        plot_position_of_trials(data,speed,ax)
    
    fig.tight_layout()
    pdf.savefig(fig)
    
pdf.close()
clear_output()

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages("/home/david/Documents/TEMP/Lesion.pdf")
#Lesion
for i,session in enumerate(sessionList1):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    fig=plt.figure(figsize=(15,5*len(_SPD_)))
    for i,speed in enumerate(_SPD_):
        ax=fig.add_subplot(len(_SPD_),1,i+1)
        plot_position_of_trials(data,speed,ax)
    
    fig.tight_layout()
    pdf.savefig(fig)
    
pdf.close()
clear_output()

# ViewPoint intertrial analysis for Loco test

check normal exploration

In [ ]:

def find_persistent_movement_epochs(pos: np.array, TrdSpeed: int, fs: int):
    """
    this function finds epochs in the position of a single trial, pos, where the animal has moved
    for a duration of at least, TH*cameraSamplingRate
    pos: position as a np.array
    """
    SpeedTolerance = 5 if TrdSpeed==0 else 10
    DurationTolerance= 0.4*fs
    
    pos_=smooth(pos,.3*fs,mode="nearest")
    velocity=np.append(TrdSpeed-(np.diff(pos_)*fs),TrdSpeed)
    
    zeroVel=np.logical_and(velocity<SpeedTolerance, velocity>-SpeedTolerance)
    
    movement= np.logical_not(zeroVel)
    movement=movement*1

    dpcross=np.absolute( np.append(movement,0) - np.append(0,movement) )
#     breakpoint()
    #any 1 in dpcross mean a zero-crossing in diff(pos)
    crossindex=np.where(dpcross)[0]
    crossindex=np.reshape(crossindex,(-1,2))
    movementepoch=crossindex[crossindex[:,1]-crossindex[:,0] > DurationTolerance]
    return movementepoch

def run_distance(data,trial=None):
    if trial is None:
        trial=list(data.position.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    assert data.dataType == 'ViewPoint',f'{data.dataType}'
    
    dis=[0]
    for i in trial:
        if i < 10:
            continue #first 10 session with short trial
        allTraj=data.position[i]
        treadmillStep=np.expand_dims(data.treadmillSpeed[i]/data.cameraSamplingRate,1)
        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TrdSpeed=data.treadmillSpeed[i],
                                                     fs=data.cameraSamplingRate)

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)-treadmillStep))
        dis.append(0)

    return np.array(dis[:-1])

def intertrial_displacement(data,trial=None):
    if trial is None:
        trial=list(data.rawIntertrialPosition.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    dis=[0]
    for i in trial:
        if data.dataType != 'ViewPoint':
            allTraj= data.position[i][int((-data.interTrialDuration[i]+1)*data.cameraSamplingRate):]
            TH=int(0.6*data.cameraSamplingRate)
            data.cameraSamplingRateInterTrial=data.cameraSamplingRate
        else:
            allTraj=smooth(data.rawIntertrialPosition[i],.3*data.cameraSamplingRateInterTrial)
            TH=int(0.6*data.cameraSamplingRateInterTrial)
            

        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TrdSpeed=0,
                                                     fs=data.cameraSamplingRateInterTrial
                                                     )

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)))
        dis.append(0)

    return np.array(dis[:-1])

In [ ]:
def intertrial_max_velocity(data,sigmaSpeed=0.3):
    
    cs=data.cameraSamplingRateInterTrial
    tr=0
    
    vel=[]
    for i,pos in data.rawIntertrialPosition.items():
        speed= tr - np.diff(pos)*cs 
        speed=smooth(speed,sigmaSpeed*cs)
        vel.append(np.nanmax(np.absolute(np.append(speed,np.nan))))
    
    return np.array(vel)


def trial_max_velocity(data,sigmaSpeed=0.3):
    cs=data.cameraSamplingRate
    
    vel=np.ones((list(data.position.keys())[-1]+1,))
    for i,pos in data.position.items():
        tr=data.treadmillSpeed[i]
        speed= tr - np.diff(pos)*cs 
        speed=smooth(speed,sigmaSpeed*cs)
        vel[i]=np.nanmax(np.absolute(np.append(speed,np.nan)))
    
    return vel

def trial_max_acceleration(data,sigmaSpeed=0.3):
    cs=data.cameraSamplingRate
    
    acc=np.ones((list(data.position.keys())[-1]+1,))
    for i,pos in data.position.items():
        tr=data.treadmillSpeed[i]
        speed= tr - np.diff(pos)*cs 
        speed=smooth(speed,sigmaSpeed*cs)
        accelaration=np.diff(speed)*cs
        acc[i]=np.nanmax(np.absolute(np.append(accelaration,np.nan)))
    
    return acc

In [ ]:
def time_in_back(data):    
    assert data.dataType == 'ViewPoint'
    cs=data.cameraSamplingRate
    t=[]
    for i in data.position.keys():
        if i < 10:
            continue #first 10 session with short trial
        traj=data.position[i]
        t.append(np.sum(traj>75)/cs)

    return np.array(t)

def position_variability(data):
    assert data.dataType == 'ViewPoint'
    cs=data.cameraSamplingRate
    t=[]
    for i in data.position.keys():
        if i < 10:
            continue #first 10 session with short trial
        traj=data.position[i]
        t.append(np.std(traj))

    return np.array(t)


In [ ]:
disIT0=np.ones((45,len(sessionList0)))*np.nan
disIT1=np.ones((45,len(sessionList1)))*np.nan

dis0=np.ones((45,len(sessionList0)))*np.nan
dis1=np.ones((45,len(sessionList1)))*np.nan

NbTrial0=np.ones((len(sessionList0),))*np.nan
NbTrial1=np.ones((len(sessionList1),))*np.nan

velIT0=np.ones((45,len(sessionList0)))*np.nan
velIT1=np.ones((45,len(sessionList1)))*np.nan

vel0=np.ones((45,len(sessionList0)))*np.nan
vel1=np.ones((45,len(sessionList1)))*np.nan

acc0=np.ones((45,len(sessionList0)))*np.nan
acc1=np.ones((45,len(sessionList1)))*np.nan


Ftime0=np.ones((45,len(sessionList0)))*np.nan
Ftime1=np.ones((45,len(sessionList1)))*np.nan

posSD0=np.ones((45,len(sessionList0)))*np.nan
posSD1=np.ones((45,len(sessionList1)))*np.nan



for i,session in enumerate(sessionList0):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    tmp=intertrial_displacement(data)
    disIT0[:len(tmp),i]=tmp
    tmp=run_distance(data)
    dis0[:len(tmp),i]=tmp
    tmp=intertrial_max_velocity(data)
    velIT0[:len(tmp),i]=tmp
    tmp=trial_max_velocity(data)
    vel0[:len(tmp),i]=tmp
    tmp=time_in_back(data)
    Ftime0[:len(tmp),i]=tmp
    tmp=position_variability(data)
    posSD0[:len(tmp),i]=tmp
    tmp=trial_max_acceleration(data)
    acc0[:len(tmp),i]=tmp

    

    
    NbTrial0[i]=data.nTrial

for i,session in enumerate(sessionList1):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    tmp=intertrial_displacement(data)
    disIT1[:len(tmp),i]=tmp
    tmp=run_distance(data)
    dis1[:len(tmp),i]=tmp
    tmp=intertrial_max_velocity(data)
    velIT1[:len(tmp),i]=tmp
    tmp=trial_max_velocity(data)
    vel1[:len(tmp),i]=tmp
    tmp=time_in_back(data)
    Ftime1[:len(tmp),i]=tmp
    tmp=position_variability(data)
    posSD1[:len(tmp),i]=tmp
    tmp=trial_max_acceleration(data)
    acc1[:len(tmp),i]=tmp



    NbTrial1[i]=data.nTrial


# Position variability during Trials

In [ ]:
a=posSD0[:,:]
b=posSD1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(posSD0,'k');
plt.plot(posSD1,'r');

# Time in the back

In [ ]:
a=Ftime0[:,:]
b=Ftime1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
plt.plot(Ftime0,'k');
plt.plot(Ftime1,'r');

# Displacement

In [ ]:
a=disIT0[:,:]
b=disIT1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
a=dis0[:,:]
b=dis1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
plt.plot(dis0,'k');
plt.plot(dis1,'r');

Checking whether groups differ in their max trial number

In [ ]:
plt.plot(NbTrial0)
plt.plot(NbTrial1)
plt.show()

In [ ]:
perm=permtest(NbTrial0,NbTrial1,nIterations=10000)

perm.pVal

#  velocity

In [ ]:
#INTERTRIALS
a=velIT0[:,:]
b=velIT1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
#TRIALS
a=vel0[:,:]
b=vel1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
plt.plot(vel0,'k');
plt.plot(vel1,'r');

# Max Trial Accelaration

In [ ]:
#TRIALS
a=acc0[:,:]
b=acc1[:,:]

perm=permtest(a.T,b.T,nIterations=10000)

perm.pVal

In [ ]:
plt.plot(acc0,'k');
plt.plot(acc1,'r');

# Weight Comparison

In [ ]:
CtrlW   =dict.fromkeys(NormalAnimalList,np.nan)
LesionW =dict.fromkeys(LesionAnimalList,np.nan)
Weights=dict()

### Data From Mostafa's Lab Book

In [ ]:
Weights['Rat367']=560
Weights['Rat368']=500
Weights['Rat372']=435
Weights['Rat366']=450
Weights['Rat371']=425
Weights['Rat369']=505
Weights['Rat365']=500
Weights['Rat349']=440
Weights['Rat373']=380
Weights['Rat358']=490
Weights['Rat357']=445
Weights['Rat356']=450
Weights['Rat361']=440
Weights['Rat360']=460
Weights['Rat359']=475
Weights['Rat364']=335
Weights['Rat363']=385
Weights['Rat362']=420
Weights['Rat355']=400
Weights['Rat354']=340
Weights['Rat353']=385

In [ ]:
for animal,w in Weights.items():
    if animal in CtrlW:
        CtrlW.update({animal:w})
    elif animal in LesionW:
        LesionW.update({animal:w})
        
print(CtrlW,'\n',LesionW)

# Plot Speed distributions

In [ ]:
def intertrial_VUpoint_velocity(data,trial,sigmaSpeed=0.3):
    
    cs=data.cameraSamplingRateInterTrial
    tr=0
    
    if trial not in data.rawIntertrialPosition or len(data.rawIntertrialPosition[trial])<2:
        return np.array([0])
    
    pos=data.rawIntertrialPosition[trial]
    speed= tr - np.diff(pos)*cs 
    speed=smooth(speed,sigmaSpeed*cs)
        
    return np.absolute(speed)


def trial_VUpoint_velocity(data,trial,sigmaSpeed=0.3):
    cs=data.cameraSamplingRate
    
    if trial not in data.position or len(data.position[trial])<2:
        return np.array([0])
    
    pos=data.position[trial]
    tr=data.treadmillSpeed[trial]
    speed= tr - np.diff(pos)*cs 
    speed=smooth(speed,sigmaSpeed*cs)

    return np.absolute(speed)


In [ ]:
# TrialG=(19,20,21)   #speed=20
TrialG=(25,26,27,28,29)  #speed=30

spdG0={}
spdG1={}

for i,session in enumerate(sessionList0):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    spdG0[animal]=np.array([])
    
    for trial in TrialG:
        spdG0[animal]=np.concatenate((spdG0[animal],trial_VUpoint_velocity(data,trial)))

        
for i,session in enumerate(sessionList1):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    spdG1[animal]=np.array([])
    
    for trial in TrialG:
        spdG1[animal]=np.concatenate((spdG1[animal],trial_VUpoint_velocity(data,trial)))
        
speedDict={**spdG0,**spdG1}

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("/home/david/Documents/TEMP/T30.pdf")
for animal,data in spdG0.items():
    f=plt.figure()
    ax=f.add_subplot()
    
    ax.hist(data,100)
    ax.set_title(animal)
    ax.set_xlim([-10,200])
    
    pdf.savefig(f)

pdf.close()

# fast movement fraction

In [ ]:
def trial_fast_velocity_percentage(data,trials,THspeed=50,sigmaSpeed=0.3):
    cs=data.cameraSamplingRate
    
    vel=np.array([])
    for trial in trials:
        if trial not in data.position or len(data.position[trial])<2:
            continue

        pos=data.position[trial]
        tr=data.treadmillSpeed[trial]
        speed= tr - np.diff(pos)*cs 
        speed=smooth(speed,sigmaSpeed*cs)
        vel=np.concatenate((vel,speed))
    
    fastfrac=np.sum(vel>THspeed)/len(vel)
    return fastfrac*100

In [ ]:
trdSpeed={10:(15,16),
         15:(17,18),
         20:(19,20,21),
         25:(22,23,24),
         30:(25,26,27,28,29),
#          35:(30,31,32,33,34),
#          40:(35,36,37,38,39)
         }

# trdSpeed={10:(15,16,17,18),
#           30:(25,26,27,28,29,30,31,32,33,34,35,36,37,38,39)
#          }


ControlFastPerc=np.ones( ( len(trdSpeed),len(sessionList0) ) )*np.nan
for i,session in enumerate(sessionList0):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    
    for j,(spd,trials) in enumerate(trdSpeed.items()):
        ControlFastPerc[j,i]=trial_fast_velocity_percentage(data,trials)
        
        
LesionFastPerc=np.ones( ( len(trdSpeed),len(sessionList1) ) )*np.nan
for i,session in enumerate(sessionList1):
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    
    for j,(spd,trials) in enumerate(trdSpeed.items()):
        LesionFastPerc[j,i]=trial_fast_velocity_percentage(data,trials)

In [ ]:
plt.plot(list(trdSpeed.keys()),LesionFastPerc,'r');
plt.plot(list(trdSpeed.keys()),ControlFastPerc,'k');
plt.xticks(list(trdSpeed.keys()));

In [ ]:
a=permtest(LesionFastPerc.T,ControlFastPerc.T,nIterations=10000)

a.pVal

In [ ]:
for i,d in enumerate(LesionFastPerc.T):
    plt.title(sessionList1[i])
    plt.plot(list(trdSpeed.keys()),d,'-*')
    plt.ylim([0,25])
    plt.show()
    breakpoint()
    clear_output()


In [ ]:
HistologyExcel('/NAS02','Rat354').lesion_size()

# First Exhausted Intertrial
From Mostafa's visual inspection

100 for *never*

In [ ]:
Tired=dict()
Tired['Rat367']=27
Tired['Rat368']=100
Tired['Rat372']=32
Tired['Rat366']=100
Tired['Rat371']=32
Tired['Rat369']=30
Tired['Rat365']=24
Tired['Rat349']=36
Tired['Rat373']=36
Tired['Rat358']=30
Tired['Rat357']=32
Tired['Rat356']=100
Tired['Rat361']=100
Tired['Rat360']=40
Tired['Rat359']=28
Tired['Rat325']=28
Tired['Rat345']=100
Tired['Rat341']=34
Tired['Rat337']=100
Tired['Rat363']=100
Tired['Rat362']=28
Tired['Rat355']=33
Tired['Rat354']=32
Tired['Rat353']=32

In [ ]:
controlTired=[]
for session in sessionList0:
    animal=session[:6]
    if Tired[animal]==100:
        controlTired.append(np.nan)
        continue
    
    data=Data(root,animal,session,param=param,redoPreprocess=False, saveAsPickle=False);
    controlTired.append(data.nTrial - Tired[animal])

    
LesionTired=[]
for session in sessionList1:
    animal=session[:6]
    if Tired[animal]==100:
        LesionTired.append(np.nan)
        continue
    
    data=Data(root,animal,session,param=param,redoPreprocess=False, saveAsPickle=False);
    LesionTired.append(data.nTrial - Tired[animal])


In [ ]:
controlTired,LesionTired

---
---
---
------
---
---
---

In [ ]:
session='Rat371_2019_10_30_11_36'
session='Rat362_2019_07_17_16_59'

data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);


In [ ]:
for i,pos in data.position.items():
    plt.plot(smooth(pos,.3*200))
    plt.title(f'trial {i}, speed {data.treadmillSpeed[i]}')
    plt.ylim([0,90])
    plt.show()
    breakpoint()
    clear_output()
    